In [1]:
import os
import sys
import pickle
from pyproj import CRS
import pandas as pd
import geopandas as gpd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import Scenario
from network_wrangler import WranglerLogger
from network_wrangler import ProjectCard

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc
from lasso import util

%load_ext autoreload
%autoreload 2



In [2]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.setLevel(logging.INFO)

## Remote I/O

In [3]:
pickle_dir = "../../" #Set to directory where the pickle files are stored
card_dir = "../../ProjectCards"#Set to directory where the cards are stored
transit_dir = "../../Transit"
lasso_dir = '..'
transit_project_card_dir = os.path.join(card_dir, "transit")
highway_project_card_dir = os.path.join(card_dir, '2035','1-2')

parameters = Parameters(lasso_base_dir = lasso_dir)


2024-03-15 18:49:49, INFO: Lasso base directory set as: ..
2024-03-15 18:49:49, INFO: Lasso base directory set as: ..


# Load Scenario

In [4]:
pickle_file_name = os.path.join(pickle_dir, "scenario_2020.pickle")
curr_scenario = pickle.load(open(pickle_file_name, 'rb'))

# Step 01: Apply Associal Highway Project Cards

In [5]:
curr_scenario.road_net.links_df['name']=curr_scenario.road_net.links_df['name'].apply(lambda x: "" if type(x)==int else x)
curr_scenario.road_net.links_df['name']=curr_scenario.road_net.links_df['name'].apply(lambda x: util.shorten_name(x))

In [10]:
project_card_list = []

# scenario_2015_cards = [ 
#                         'Added_networkforexternals_Sacto2_GN_11142022.yml',
#                         'assignable_and_ace_transit_connectors.yml',
#                         'CCTA.yml',
#                         'Externals_SJQ_Add_complete_network_10032022.yml',
#                         'Sonoma_Marin_Area_Rail_Transit_add_rail_link.yml',
#                         'Year 2015 Modify Express Lanes and USE on I-580 EB (segment 02) Hacienda Drive to Airway Blvd_v2.yml',
#                         'Year 2015 Modify HOV Lanes and USE on CA-237 EB US 101 to North 1st Street_v2.yml',
#                         'Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101_v2.yml',
#                         'Year 2015 Modify HOV Lanes and USE on I-680 NB from Alcosta Blvd to Livorna Road_v2.yml',
#                         'Year 2015 Modify HOV Lanes and USE on I-680 SB from Benicia Bridge to Alcosta Blvd_v2.yml',
#                         'year_2015_managed_lane_i680n_ca_242_to_benicia_bridge.yml'
#                         ]

# for card in scenario_2015_cards:
#     _filename = os.path.join(card_dir, '2020',card)
#     card = ProjectCard.read(_filename, validate = False)
#     project_card_list.append(card)

scenario_2035_cards = [
                    ### Projects in San Joaqun
                    'san_joaquin_05_sj07_1006.yml',
                    'san_joaquin_06_sj07_1005.yml',
                    'san_joaquin_07_sj11_1001.yml',
                    'san_joaquin_08_sj18_1001.yml',
                    'san_joaquin_09_sj07_1014.yml',
                    'san_joaquin_01_sj14_1001.yml',
                    'san_joaquin_02_sj14_1002.yml',
                    'san_joaquin_03_sj07_1003.yml',
                    'san_joaquin_04_sj07_1008.yml',
                    ### Projects modifying highway related to transit improvements (BRT lines, Station/Stop Updates etc.)
                    '0_01_CongestionPricingDowntownSF_21-T10-091.yml',
                    'Sonoma_Marin_Area_Rail_Transit_add_rail_link.yml', #If you are starting from 2015 pickle file, comment this card out since it is already added on the 2015 network.
                    '0_02_CongestionPricingTreasureIsland_21-T10-092.yml',
                    '0_06_SR-237 Corridor Improvement 21-T06-043.wrangler',
                    '0_08_I-680 Corridor Improvement 21-T06-022.wrangler',
                    '0_10_1_21-T06-027-FBP_SM_027_US101_92.yml',
                    '0_10_2_21-T06-027-FBP_SM_033_US101_Holly_Interchange_add.yml',
                    '0_10_3_21-T06-027-FBP_SM_033_US101_Holly_Interchange_mod.yml',
                    '0_10_4_21-T06-027-FBP_SM_035_Peninsula_101_OnOffRamps.yml',
                    '0_10_5_21-T06-027-STIP_ProduceAve.yml',
                    '0_11_1_21-T06-028-FBP_SC_072_US101_Trimble_Interchange.yml',
                    '0_11_2_21-T06-028-FBP_SC_073_BlossomHill_101Wide.yml',
                    '0_11_5_21-T06-028-FBP_SC_081_US101_SR237.wrangler',
                    '0_11_7_21-T06-028-FBP_SC_083_US101_Zanker_Skyport_Interchange.yml',
                    '0_12_21-T06-015-MAJ_SOL070020_I80_I680_SR12_Int_2B_7_mod.yml',
                    '10_21-T11-114_Transform_Valley_Link_add_rail_link.yml',
                    '2_1_14_MAJ_MTC050027_Berkeley_Ferry_add_ferry_link.yml',
                    '2_4_15_21-T11-97-MAJ_MissionBay_SF_Ferry_add_ferry_link.yml',
                    '2_5_16_21_T11-98-MAJ_RedwoodCity_SF_Ferry_add_ferry_link.yml',
                    '3_21-T10-083-Central_Subway_ToChinaTown_add_rail_link.yml',
                    '5_1_Rex_Blue_add_highway_link.yml',
                    'BART_2020_Fremont_to_Berryessa_add_rail_link.yml',
                    'BART_2036_Berryessa_to_Santa_Clara_add_rail_link.yml',
                    'Caltrain_Peninsula_Corridor_Electrification_Expansion_add_rail_link.yml',
                    'extra_brt_14th_st_mission_blvd_brt1.yml',
                    'extra_brt_albany_to_oakland_brt2.yml',
                    'extra_brt_broadway_brt1.yml',
                    'extra_brt_grand_ave_brt1.yml',
                    'extra_brt_lower_dimond_to_alameda_brt2.yml',
                    'extra_brt_marin_ave_brt2.yml',
                    'extra_brt_merritt_to_ashland_brt2.yml',
                    'extra_brt_mission_grain_to_grimmer_brt2.yml',
                    'extra_brt_oakland_brt2.yml',
                    'extra_brt_ralph_appezzato_memorial_pwky_brt1.yml',
                    'extra_brt_thomas_berkley_way_grand_ave_brt1.yml',
                    'year_2020_better_market_st_eb.yml',
                    'year_2020_better_market_st_wb.yml',
                    'year_2020_broadway_brt.yml',
                    'year_2030_i_80_wb_bus_only_hov_extension.yml',
                    'year_2020_richmond_san_rafael_bridge_access_improvement.yml',
                    'year_2020_tempo_brt.yml',
                    'year_2021_geary_brt_phase1_eb.yml',
                    'year_2021_geary_brt_phase1_wb.yml',
                    'year_2021_sr4_wb_added_gp_lane_sr242_to_i680.yml',
                    'year_2022_van_ness_brt_nb.yml',
                    'year_2022_van_ness_brt_sb.yml',
                    ### Adding new express lanes
                    'year_2023_us101_el_whipple_to_i380.yml',
                    'year_2024_san_pablo_ave_rapid_corridors_project.yml',
                    'year_2025_bay_bridge_forward_i580wb_hov_extension.yml',
                    'year_2025_i680_sr237_to_sr_84_hov_lanes_new_construction.yml',
                    'year_2025_i80_solano_managed_lanes.yml',
                    'year_2025_sr84_widening_rubyhill_to_i680.yml',
                    'year_2025_us101_marin_sonoma_narrows.yml',
                    'year_2028_geary_brt_phase2_eb.yml',
                    'year_2028_geary_brt_phase2_wb.yml',
                    'year_2029_san_pablo_ave_bus_bike_project.yml',
                    'year_2030_23rd_st_brt.yml',
                    'year_2030_broadway_streetscape_improvement_project.yml',
                    'year_2030_i280_us101_nb_hov.yml',
                    'year_2030_i280_us101_sb_hov.yml',
                    'year_2030_i580_wb_rsr_forward.yml',
                    'year_2030_i680_express_lanes_new_roadway_nb_sb.yml',
                    'year_2030_i680_nb_express_lane_completion_2.yml',
                    'year_2030_i680_nb_pttl.yml',
                    'year_2030_i680_sb_pttl.yml',
                    'year_2030_sr37_sears_point_to_mareisland_hov.yml',
                    'year_2030_sr4_wb_el_sr242_to_i680.yml',
                    'year_2030_us101_ml_north_of_i380_to_sf_sm_countyline.yml',
                    'year_2033_i680_el_calaveras_to_us101.yml',
                    'year_2035_bike_walk_access_on_bridges.yml'
                    ]

for card in scenario_2035_cards:
    _filename = os.path.join(card_dir, '2035','1-2',card)
    card = ProjectCard.read(_filename, validate = False)
    project_card_list.append(card)


In [11]:
# for item in project_card_list:
#     curr_scenario.apply_project(item)
    
step1_version_04_scenario = Scenario.create_scenario(
    base_scenario = curr_scenario,
    project_cards_list=[], 
    validate_project_cards=False
)

for item in project_card_list:
    step1_version_04_scenario.apply_project(item)

2024-03-15 19:08:09, INFO: Creating Scenario
2024-03-15 19:12:24, INFO: Applying [../../ProjectCards\2035\1-2\san_joaquin_05_sj07_1006.yml]:San Joaquin SJ07-1006
2024-03-15 19:12:24, INFO: Applying [../../ProjectCards\2035\1-2\san_joaquin_05_sj07_1006.yml]:San Joaquin SJ07-1006
2024-03-15 19:12:24, INFO: Applying Project to Roadway Network: San Joaquin SJ07-1006


c:\Users\USJH706661\.conda\envs\bcm_network\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


2024-03-15 19:12:25, INFO: Applying [../../ProjectCards\2035\1-2\san_joaquin_06_sj07_1005.yml]:San Joaquin SJ07-1005
2024-03-15 19:12:25, INFO: Applying [../../ProjectCards\2035\1-2\san_joaquin_06_sj07_1005.yml]:San Joaquin SJ07-1005
2024-03-15 19:12:25, INFO: Applying Project to Roadway Network: San Joaquin SJ07-1005


c:\Users\USJH706661\.conda\envs\bcm_network\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


2024-03-15 19:12:26, INFO: Applying [../../ProjectCards\2035\1-2\san_joaquin_07_sj11_1001.yml]:San Joaquin SJ11-1001
2024-03-15 19:12:26, INFO: Applying [../../ProjectCards\2035\1-2\san_joaquin_07_sj11_1001.yml]:San Joaquin SJ11-1001
2024-03-15 19:12:26, INFO: Applying Project to Roadway Network: San Joaquin SJ11-1001


...

c:\Users\USJH706661\.conda\envs\bcm_network\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


2024-03-15 20:00:31, INFO: Applying [../../ProjectCards\2035\1-2\year_2033_i680_el_calaveras_to_us101.yml]:Year 2033 Modify GP Lanes and USE on I-680 Calaveras Blvd (SR-237) to US-101
2024-03-15 20:00:31, INFO: Applying [../../ProjectCards\2035\1-2\year_2033_i680_el_calaveras_to_us101.yml]:Year 2033 Modify GP Lanes and USE on I-680 Calaveras Blvd (SR-237) to US-101
2024-03-15 20:00:31, INFO: Applying Project to Roadway Network: Year 2033 Modify GP Lanes and USE on I-680 Calaveras Blvd (SR-237) to US-101


c:\Users\USJH706661\.conda\envs\bcm_network\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


2024-03-15 20:00:33, INFO: Applying [../../ProjectCards\2035\1-2\year_2035_bike_walk_access_on_bridges.yml]:Year 2035 Bike and Walk Access on Golden Gate, Dumberton, ANtioch, Bay Bridge East Span, San Rafael Bridges
2024-03-15 20:00:33, INFO: Applying [../../ProjectCards\2035\1-2\year_2035_bike_walk_access_on_bridges.yml]:Year 2035 Bike and Walk Access on Golden Gate, Dumberton, ANtioch, Bay Bridge East Span, San Rafael Bridges
2024-03-15 20:00:33, INFO: Applying Project to Roadway Network: Year 2035 Bike and Walk Access on Golden Gate, Dumberton, ANtioch, Bay Bridge East Span, San Rafael Bridges
2024-03-15 20:00:33, INFO: Applying [../../ProjectCards\2035\1-2\year_2035_bike_walk_access_on_bridges.yml]:Year 2035 Bike and Walk Access on Golden Gate, Dumberton, ANtioch, Bay Bridge East Span, San Rafael Bridges
2024-03-15 20:00:33, INFO: Applying Project to Roadway Network: Year 2035 Bike and Walk Access on Golden Gate, Dumberton, ANtioch, Bay Bridge East Span, San Rafael Bridges


c:\Users\USJH706661\.conda\envs\bcm_network\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [12]:
step1_version_04_scenario.road_net.links_df = gpd.GeoDataFrame(step1_version_04_scenario.road_net.links_df, geometry=step1_version_04_scenario.road_net.links_df.geometry)
step1_version_04_scenario.road_net.nodes_df = gpd.GeoDataFrame(step1_version_04_scenario.road_net.nodes_df, geometry=step1_version_04_scenario.road_net.nodes_df.geometry)
step1_version_04_scenario.transit_net.set_roadnet(step1_version_04_scenario.road_net, validate_consistency=False)

# Step 02: Apply Transit Project Cards

In [13]:
step2_version_04_scenario = Scenario.create_scenario(
    base_scenario = step1_version_04_scenario,
    card_directory = transit_project_card_dir,
    project_cards_list=[],
    validate_project_cards=False
)
step2_version_04_scenario.apply_all_projects()


2024-03-15 20:04:26, INFO: Creating Scenario
2024-03-15 20:10:42, ERROR: Projects 10_21-t11-114_transform_valley_link has 10_21-t11-114_transform_valley_link_add_rail_link as prerequisite project which is missing for the scenario
2024-03-15 20:10:42, ERROR: Projects 2_1_14_maj_mtc050027_berkeley_ferry has 2_1_14_maj_mtc050027_berkeley_ferry_add_ferry_link as prerequisite project which is missing for the scenario
2024-03-15 20:10:42, ERROR: Projects 2_4_15_21-t11-97-maj_missionbay_sf_ferry has 2_4_15_21-t11-97-maj_missionbay_sf_ferry_add_ferry_link as prerequisite project which is missing for the scenario
2024-03-15 20:10:42, ERROR: Projects 2_5_16_21_t11-98-maj_redwoodcity_sf_ferry has 2_5_16_21_t11-98-maj_redwoodcity_sf_ferry_add_ferry_link as prerequisite project which is missing for the scenario
2024-03-15 20:10:42, ERROR: Projects 3_21-t10-083-central_subway_tochinatown_modify_tline_shape has 3_21-t10-083-central_subway_tochinatown_add_rail_link as prerequisite project which is mis

In [ ]:
# in case any transit shapes are broken because of the roadway network changes
# step2_version_04_scenario.update_transit_net_with_new_road_net()

# Write model network for Cube

In [14]:
standard_transit_net = StandardTransit.fromTransitNetwork(step2_version_04_scenario.transit_net, parameters=parameters)
mtc.write_as_cube_lin(standard_transit_net, parameters, outpath = os.path.join(transit_dir, "2035", "transit.lin"))

2024-03-15 20:31:01, INFO: Lasso base directory set as: ..
2024-03-15 20:31:01, INFO: Lasso base directory set as: ..
2024-03-15 20:31:01, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
